In [1]:
import pandas as pd
import os
%config Completer.use_jedi = False
csvpath = os.path.join("HeroesOfPymoli","Resources","purchase_data.csv")

purchase_data = pd.read_csv(csvpath)

pymoli_df = pd.DataFrame(purchase_data)

pymoli_df.head()


,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [2]:
player_count_df = pymoli_df["SN"].nunique()
playerdict = [{"Total Players" : (player_count_df)}]
totalplayers = pd.DataFrame(playerdict)
totalplayers

,Total Players
0,576


In [3]:
#Count of unique items
item_types_df = pymoli_df["Item ID"].nunique()

#Calculate average purchase price
ave_price = pymoli_df["Price"].mean()
ave_price_currency = "${:.2f}".format(ave_price)
#Total number of purchases
num_purchases = pymoli_df["Purchase ID"].count()

#calculate total Revenue
total_revenue = pymoli_df["Price"].sum()
total_revenue_currency = "${:.2f}".format(total_revenue)
#create dictionary
purchasing_analysis_dict = [{"Number of Unique Items" : (item_types_df), "Average Price" : (ave_price_currency), "Number of Purchases": (num_purchases), "Total Revenue" : (total_revenue_currency)}]

purchasing_analysis_df = pd.DataFrame(purchasing_analysis_dict)

purchasing_analysis_df


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


In [8]:
#gender demos and total percentage
player_count = len(purchase_data["SN"].unique())

#players
male_players = purchase_data.loc[purchase_data["Gender"] == "Male"]
male_count = len(male_players["SN"].unique())
male_percent = "{:.2f}%".format(male_count / player_count * 100)

female_players = purchase_data.loc[purchase_data["Gender"] == "Female"]
female_count = len(female_players["SN"].unique())
female_percent = "{:.2f}%".format(female_count / player_count * 100)

other_players = purchase_data.loc[purchase_data["Gender"] == "Other / Non-Disclosed"]
other_count = len(other_players["SN"].unique())
other_percent = "{:.2f}%".format(other_count / player_count * 100)

#summary
gender_demographics_table = pd.DataFrame([{
    "Gender": "Male", "Total Count": male_count, 
    "Percentage of Players": male_percent}, 
    {"Gender": "Female", "Total Count": female_count, 
     "Percentage of Players": female_percent}, 
    {"Gender": "Other / Non-Disclosed", "Total Count": other_count, 
     "Percentage of Players": other_percent
    }], columns=["Gender", "Total Count", "Percentage of Players"])

gender_demographics_table = gender_demographics_table.set_index("Gender")
gender_demographics_table.index.name = "Gender"
gender_demographics_table



,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [11]:
#create grouped data frame
gender_grouped_df = pymoli_df.groupby(["Gender"])
gpurchase_count = gender_grouped_df["Purchase ID"].count()
gprice_ave = gender_grouped_df["Price"].mean().map("${:.2f}".format)
gtotal_value =gender_grouped_df["Price"].sum().map("${:.2f}".format)

gendertotals = gender_grouped_df["Price"].sum()
individuals = gender_grouped_df["SN"].nunique()
ave_per_player = (gendertotals/individuals).map("${:.2f}".format)

gender_analysis_df = pd.DataFrame({"Purchase Count" : gpurchase_count, "Average Purchase Price" : gprice_ave, "Total Purchase Value" : gtotal_value, "Avg Total Purchase Per Person" : ave_per_player})

gender_analysis_df





,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase Per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56
